In [1]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Fri Apr 14 15:38:06 2017

@author: zhouyi
"""

import glob
import os
import sys

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.sparse
import pandas as pd

import expomf

In [2]:
DATA_ROOT = '/Users/zhouyi/Documents/GraduatePJ/MSD'

unique_uid = list()
with open(os.path.join(DATA_ROOT, 'unique_uid_sub.txt'), 'r') as f:
    for line in f:
        unique_uid.append(line.strip())
        
unique_sid = list()
with open(os.path.join(DATA_ROOT, 'unique_sid_sub.txt'), 'r') as f:
    for line in f:
        unique_sid.append(line.strip())

n_songs = len(unique_sid)
n_users = len(unique_uid)

In [3]:
#Load the data and train the model
def load_data(csv_file, shape=(n_users, n_songs)):
    tp = pd.read_csv(csv_file)
    rows, cols = np.array(tp['uid'], dtype=np.int32), np.array(tp['sid'], dtype=np.int32)
    count = tp['count']
    return scipy.sparse.csr_matrix((count,(rows, cols)), dtype=np.int16, shape=shape), rows, cols

train_data, rows, cols = load_data(os.path.join(DATA_ROOT, 'train.num.sub.csv'))
## binarize the data
train_data.data = np.ones_like(train_data.data)
print 'train_data.shape:',train_data.shape
print 'train_data entries:',train_data.data.shape

vad_data, rows_vad, cols_vad = load_data(os.path.join(DATA_ROOT, 'vad.num.sub.csv'))
## binarize the data
vad_data.data = np.ones_like(vad_data.data)
print 'vad__data.shape:',vad_data.shape
print 'vad_data entries:',vad_data.data.shape
test_data, rows_test, cols_test = load_data(os.path.join(DATA_ROOT, 'test.num.sub.csv'))
## binarize the data
test_data.data = np.ones_like(test_data.data)
print 'test_data.shape:',test_data.shape
print 'test_data entries:',test_data.data.shape
print 'sparse according to vad_data:',float(vad_data.data.shape[0])/(vad_data.shape[0]*vad_data.shape[1])

train_data.shape: (1013, 400)
train_data entries: (24587,)
vad__data.shape: (1013, 400)
vad_data entries: (2731,)
test_data.shape: (1013, 400)
test_data entries: (6829,)
sparse according to vad_data: 0.00673988153998


In [5]:
n_components = 25
max_iter = 20
n_jobs = 10
# grid search on init_mu = {0.1, 0.05, 0.01, 0.005, 0.001} from validation set
init_mu = 0.0067
lam = 1e-5

save_dir="ExpoMF_TSP_K%d_lam%1.0E_initmu%1.0E" % (n_components, lam, init_mu)

coder = expomf.ExpoMF(n_components=n_components, max_iter=max_iter, batch_size=10, init_std=0.01, 
                      n_jobs=n_jobs, random_state=98765, save_params=True, 
                      save_dir=save_dir, early_stopping=True, verbose=True, 
                      lam_y=1., lam_theta=lam, lam_beta=lam, a=1., b=1., init_mu=init_mu)

coder.fit(train_data, vad_data=vad_data, k=20)

ITERATION #0
	Updating user factors: time=0.50
	Updating item factors: time=0.54
	Updating exposure prior: time=0.05
	Validation NDCG@k: 0.0106
ITERATION #1
	Updating user factors: time=0.44
	Updating item factors: time=0.43
	Updating exposure prior: time=0.06
	Validation NDCG@k: 0.0153
ITERATION #2
	Updating user factors: time=0.58
	Updating item factors: time=0.53
	Updating exposure prior: time=0.06
	Validation NDCG@k: 0.0163
ITERATION #3
	Updating user factors: time=0.44
	Updating item factors: time=0.43
	Updating exposure prior: time=0.06
	Validation NDCG@k: 0.0172
ITERATION #4
	Updating user factors: time=0.68
	Updating item factors: time=0.63
	Updating exposure prior: time=0.09
	Validation NDCG@k: 0.0185
ITERATION #5
	Updating user factors: time=0.43
	Updating item factors: time=0.47
	Updating exposure prior: time=0.07
	Validation NDCG@k: 0.0192
ITERATION #6
	Updating user factors: time=0.51
	Updating item factors: time=0.45
	Updating exposure prior: time=0.07
	Validation NDCG@k:

ExpoMF(batch_size=10, early_stopping=True, init_std=0.01, max_iter=20,
    n_components=25, n_jobs=10, random_state=98765,
    save_dir='ExpoMF_TSP_K25_lam1E-05_initmu7E-03', save_params=True,
    verbose=True)